In [1]:
import os, sys

fp_repo = '/Users/rfouyang/workspace/services/ds-fintech'
if fp_repo not in sys.path:
    sys.path.insert(0, fp_repo)

In [2]:
from pathlib import Path
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import toad

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from util import woe_helper
from util import woe_mono
from util import scorebin_helper
from util import report_helper

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# 加载数据

In [6]:
fp_base = Path('/Users', 'rfouyang', 'workspace')
#fp_base = Path('C:\\', 'Users', 'USER', 'workspace')
fp_data = Path(fp_base, 'data', 'data.csv')

df_data = pd.read_csv(fp_data, index_col=None)

In [7]:
num_cols = ['Collateral_valuation', 'Age', 'Properties_Total', 'Amount', 'Term', 'Historic_Loans', 'Current_Loans', 'Max_Arrears']
cat_cols = ['Region', 'Area', 'Activity', 'Guarantor', 'Collateral', 'Properties_Status']
features = num_cols + cat_cols
label = 'Defaulter'

In [8]:
for c in cat_cols:
    df_data.loc[:, c] = df_data.loc[:, c].apply(lambda x: str(f"cat_{x}"))

# 获得分箱和编码

In [9]:
woe = woe_helper.WOE()
woe.fit(df_data, label, exclude=['AppNo', label], method='dt')

100%|███████████████████████████████████████████| 14/14 [00:00<00:00, 41.21it/s]


finish combiner fit
finish combiner transform


100%|███████████████████████████████████████████| 14/14 [00:00<00:00, 46.01it/s]


finish WOE fit


In [10]:
df_woe = woe.transform(df_data)
df_bin = woe.transform(df_data, bin_only=True)

100%|███████████████████████████████████████████| 14/14 [00:00<00:00, 82.93it/s]


# 特征统计报表和区分度报表

In [11]:
df_report = report_helper.FTReport.get_report(df_bin, features, label)

In [12]:
df_eval = report_helper.FTReport.eval_metrics(df_woe, features, label)

In [13]:
df_combined_report = df_report.merge(df_eval, on='feature', how='left')
df_combined_report

,feature,bin,total_pct,total,bad,bad_rate,iv,auc,ks
0,Collateral_valuation,00.[-inf ~ 2057.0),0.15534,7767,657,0.084589,0.157705,0.602955,0.161100
1,Collateral_valuation,01.[2057.0 ~ 4571.0),0.17190,8595,988,0.114951,0.157705,0.602955,0.161100
2,Collateral_valuation,02.[4571.0 ~ 8324.5),0.11508,5754,884,0.153632,0.157705,0.602955,0.161100
3,Collateral_valuation,03.[8324.5 ~ 13823.5),0.06604,3302,604,0.182919,0.157705,0.602955,0.161100
4,Collateral_valuation,04.[13823.5 ~ inf),0.06090,3045,891,0.292611,0.157705,0.602955,0.161100
...,...,...,...,...,...,...,...,...,...
67,None,None,NaN,None,None,NaN,NaN,NaN,NaN
68,Properties_Status,00.cat_B,0.63996,31998,4522,0.141321,0.070845,0.562709,0.107431
69,Properties_Status,01.cat_D,0.22186,11093,1935,0.174434,0.070845,0.562709,0.107431
70,Properties_Status,"02.cat_A,cat_C",0.13818,6909,1762,0.255030,0.070845,0.562709,0.107431
